In [7]:
import numpy as np
import pandas as pd
from sklearn.cross_decomposition import PLSRegression

In [2]:
des_concat_train = np.load('./features/descriptions_train_concat_embed_vectors.npy')
des_concat_test = np.load('./features/descriptions_test_concat_embed_vectors.npy')
des_long_train = np.load('./features/descriptions_train_long_embed_vectors.npy')
des_long_test = np.load('./features/descriptions_test_long_embed_vectors.npy')
image_1000_train = np.load('./features/image_features_1000_train.npy')
image_1000_test = np.load('./features/image_features_1000_test.npy')
image_2048_train = np.load('./features/image_features_2048_train.npy')
image_2048_test = np.load('./features/image_features_2048_test.npy')

In [3]:
def get_nearest_20(vec, vec_set):
    dis = [np.linalg.norm(np.array(vec) - np.array(x)) for x in vec_set]
    return np.argsort(dis)[:20]

def map20score(y, pred):
    if y in pred:
        return (20 - pred.tolist().index(y)) / 20.0
    else:
        return 0

def map20eval(pls, des_vectors, image_vectors):
    image_pred = pls.predict(des_vectors)
    top_20 = [get_nearest_20(vec, image_vectors) for vec in image_pred]
    print(len(top_20))
    scores = [map20score(i, top_20[i]) for i in range(len(top_20))]
    print "score: %f" % np.mean(scores)
    return scores

In [4]:
pls_concat_1000 = PLSRegression(n_components=256)
pls_concat_1000.fit(des_concat_train, image_1000_train)

/anaconda2/lib/python2.7/site-packages/sklearn/cross_decomposition/pls_.py:77: UserWarning: Maximum number of iterations reached
  warnings.warn('Maximum number of iterations reached')


PLSRegression(copy=True, max_iter=500, n_components=256, scale=True,
       tol=1e-06)

In [6]:
pred_concat_1000 = pls_concat_1000.predict(des_concat_test)
top_20_concat_1000 = [get_nearest_20(vec, image_1000_test) for vec in pred_concat_1000]

In [12]:
def output_submission(top_20, output_path):
    top_20_image_IDs = map(lambda x: ' '.join(map(lambda xx: str(xx) + '.jpg', x)), top_20)
    description_ID = map(lambda x: str(x) + '.txt', range(len(top_20)))
    submission_df = pd.DataFrame({'Descritpion_ID': description_ID, 'Top_20_Image_IDs': top_20_image_IDs})
    submission_df.to_csv(output_path, index=False)


In [13]:
output_submission(top_20_concat_1000, 'submissions/PLSR_256_concat_1000.csv')

In [14]:
pls_concat_1000 = PLSRegression(n_components=256, max_iter=1000)
pls_concat_1000.fit(des_concat_train, image_1000_train)

PLSRegression(copy=True, max_iter=1000, n_components=256, scale=True,
       tol=1e-06)

In [15]:
pred_concat_1000 = pls_concat_1000.predict(des_concat_test)
top_20_concat_1000 = [get_nearest_20(vec, image_1000_test) for vec in pred_concat_1000]


In [16]:
output_submission(top_20_concat_1000, 'submissions/PLSR_256_concat_1000_1000iter.csv')